In [86]:
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 
import lightgbm 
import gc
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from tqdm import tqdm_notebook
from sklearn.ensemble import RandomForestRegressor
from itertools import product
from sklearn.ensemble import GradientBoostingRegressor

import matplotlib.pyplot as plt
%matplotlib inline 


for p in [np, pd, scipy, sklearn, lightgbm]:
    print (p.__name__, p.__version__)

numpy 1.13.1
pandas 0.20.1
scipy 0.19.0
sklearn 0.18.1
lightgbm 2.1.0


In [27]:

pd.set_option('display.max_rows', 600)
pd.set_option('display.max_columns', 50)

def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [28]:
sales = pd.read_csv('sales_train.csv.gz')
shops = pd.read_csv('shops.csv')
items = pd.read_csv('items.csv')
item_cats = pd.read_csv('item_categories.csv')

In [29]:
sales = sales[sales['shop_id'].isin([26, 27, 28])]

In [30]:
sales.head(), shops.head()

(             date  date_block_num  shop_id  item_id  item_price  item_cnt_day
 15036  05.01.2013               0       28     7738       199.0           1.0
 15037  07.01.2013               0       28     7738       199.0           1.0
 15038  19.01.2013               0       28     7738       199.0           1.0
 15039  03.01.2013               0       28     7737       199.0           1.0
 15040  04.01.2013               0       28     7737       199.0           1.0,
                         shop_name  shop_id
 0   !Якутск Орджоникидзе, 56 фран        0
 1   !Якутск ТЦ "Центральный" фран        1
 2                Адыгея ТЦ "Мега"        2
 3  Балашиха ТРК "Октябрь-Киномир"        3
 4        Волжский ТЦ "Волга Молл"        4)

In [31]:
index_cols = ['shop_id', 'item_id', 'date_block_num']
grid = []
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num ,'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num']==block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]]))))
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)
grid.head()

,shop_id,item_id,date_block_num
0,28,7738,0
1,28,7737,0
2,28,7770,0
3,28,7664,0
4,28,7814,0


In [32]:
gb = sales.groupby(index_cols, as_index=False).agg({'item_cnt_day':{'target':'sum'}})
gb.columns = [ col[0] if col[-1] == '' else col[-1]  for col in gb.columns.values]
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)

gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_shop':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

gb = sales.groupby(['item_id', 'date_block_num'], as_index=False).agg({'item_cnt_day':{'target_item':'sum'}})
gb.columns = [ col [0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb , how = 'left', on = ['item_id', 'date_block_num']).fillna(0)

all_data = downcast_dtypes(all_data)

C:\Users\girishgupta\Anaconda3\lib\site-packages\pandas\core\groupby.py:3961: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [33]:
del gb, grid
gc.collect();

In [34]:
cols_to_rename = list(all_data.columns.difference(index_cols)) 
shift_range = [1,2,3,4,5,12]
for month_shift in tqdm_notebook(shift_range):
    train_shift = all_data[index_cols + cols_to_rename].copy()
    train_shift['date_block_num'] = all_data['date_block_num'] + month_shift
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns = foo)
    all_data = pd.merge(all_data, train_shift, how = 'left',on= index_cols).fillna(0)

del train_shift

# Don't use old data from year 2013
all_data = all_data[all_data['date_block_num'] >= 12] 

# List of all lagged features
fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]] 
# We will drop these at fitting stage
to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 

# Category for each item
item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

all_data = pd.merge(all_data, item_category_mapping, how='left', on='item_id')
all_data = downcast_dtypes(all_data)
gc.collect();

In [35]:
all_data.head()

,shop_id,item_id,date_block_num,target,target_shop,target_item,target_lag_1,target_item_lag_1,target_shop_lag_1,target_lag_2,target_item_lag_2,target_shop_lag_2,target_lag_3,target_item_lag_3,target_shop_lag_3,target_lag_4,target_item_lag_4,target_shop_lag_4,target_lag_5,target_item_lag_5,target_shop_lag_5,target_lag_12,target_item_lag_12,target_shop_lag_12,item_category_id
0,28,10994,12,1.0,6949.0,1.0,0.0,1.0,8499.0,0.0,1.0,6454.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37
1,28,10992,12,3.0,6949.0,4.0,3.0,7.0,8499.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7521.0,0.0,0.0,0.0,37
2,28,10991,12,1.0,6949.0,5.0,1.0,3.0,8499.0,0.0,0.0,0.0,0.0,1.0,5609.0,0.0,2.0,6753.0,2.0,4.0,7521.0,0.0,0.0,0.0,40
3,28,10988,12,1.0,6949.0,2.0,2.0,5.0,8499.0,4.0,5.0,6454.0,5.0,6.0,5609.0,0.0,2.0,6753.0,0.0,0.0,0.0,0.0,0.0,0.0,40
4,28,11002,12,1.0,6949.0,1.0,0.0,1.0,8499.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40


In [36]:
to_drop_cols, fit_cols

(['target_shop', 'target', 'target_item', 'date_block_num'],
 ['target_lag_1',
  'target_item_lag_1',
  'target_shop_lag_1',
  'target_lag_2',
  'target_item_lag_2',
  'target_shop_lag_2',
  'target_lag_3',
  'target_item_lag_3',
  'target_shop_lag_3',
  'target_lag_4',
  'target_item_lag_4',
  'target_shop_lag_4',
  'target_lag_5',
  'target_item_lag_5',
  'target_shop_lag_5',
  'target_lag_12',
  'target_item_lag_12',
  'target_shop_lag_12'])

In [105]:
# Save `date_block_num`, as we can't use them as features, but will need them to split the dataset into parts 
dates = all_data['date_block_num']

last_block = dates.max()
print('Test `date_block_num` is %d' % last_block)

dates_train = dates[dates <  last_block]
dates_test  = dates[dates == last_block]

X_train = all_data.loc[dates <  last_block].drop(to_drop_cols, axis=1)
X_test =  all_data.loc[dates == last_block].drop(to_drop_cols, axis=1)

y_train = all_data.loc[dates <  last_block, 'target'].values
y_test =  all_data.loc[dates == last_block, 'target'].values

Test `date_block_num` is 33


In [71]:
lr = LinearRegression()
lr.fit(X_train.values, y_train)
pred_lr = lr.predict(X_test.values)
print('R2: ', r2_score(y_pred=pred_lr, y_true=y_test))

R2:  0.743180164534


In [72]:
lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':0 
              }
model = lgb.train(lgb_params, lgb.Dataset(X_train, label=y_train), 100)
pred_lgb = model.predict(X_test)

print('Test R-squared for LightGBM is %f' % r2_score(y_test, pred_lgb))

Test R-squared for LightGBM is 0.738969


In [89]:
xgb = GradientBoostingRegressor()
xgb.fit(X_train.values, y_train)
pred_xgb = xgb.predict(X_test.values)
print("XGB R2:" ,r2_score(y_test, pred_xgb))

XGB R2: 0.79122950531


In [106]:
X_test_level2 = np.c_[pred_lr, pred_lgb] 
X_test_l2 = np.c_[pred_lr, pred_lgb, pred_xgb]

In [107]:
dates_train_level2 = dates_train[dates_train.isin([27, 28, 29, 30, 31, 32])]

# That is how we get target for the 2nd level dataset
y_train_level2 = y_train[dates_train.isin([27, 28, 29, 30, 31, 32])]

In [108]:
# And here we create 2nd level feeature matrix, init it with zeros first
X_train_level2 = np.zeros([y_train_level2.shape[0], 2])
X_train_l2 = np.zeros([y_train_level2.shape[0], 3])
# Now fill `X_train_level2` with metafeatures
for cur_block_num in [27, 28, 29, 30, 31, 32]:
    
    print(cur_block_num)
    
    '''
        1. Split `X_train` into parts
           Remember, that corresponding dates are stored in `dates_train` 
        2. Fit linear regression 
        3. Fit LightGBM and put predictions          
        4. Store predictions from 2. and 3. in the right place of `X_train_level2`. 
           You can use `dates_train_level2` for it
           Make sure the order of the meta-features is the same as in `X_test_level2`
    '''      
    
    #  YOUR CODE GOES HERE
    dates_train = dates[dates <  cur_block_num]
    dates_test  = dates[dates == cur_block_num]

    X_train = all_data.loc[dates <  cur_block_num].drop(to_drop_cols, axis=1)
    X_test =  all_data.loc[dates == cur_block_num].drop(to_drop_cols, axis=1)

    y_train = all_data.loc[dates <  cur_block_num, 'target'].values
    y_test =  all_data.loc[dates == cur_block_num, 'target'].values
    
    
    lr.fit(X_train.values, y_train)
    pred_lr = lr.predict(X_test.values)
    print('Test R-squared for linreg is %f ' % r2_score(y_test, pred_lr))
    
    model = lgb.train(lgb_params, lgb.Dataset(X_train, label=y_train), 100)
    pred_lgb = model.predict(X_test)
    print('Test R-squared for LightGBM is %f' % r2_score(y_test, pred_lgb))
    
    xgb.fit(X_train.values, y_train)
    pred_xgb = xgb.predict(X_test.values)
    print("XGB R2:" ,r2_score(y_test, pred_xgb))
    
    #X_train_level2[] = pred_lr
    X_train_level2[dates_train_level2==cur_block_num] = np.c_[pred_lr, pred_lgb] 
    X_train_l2[dates_train_level2==cur_block_num] = np.c_[pred_lr, pred_lgb, pred_xgb] 
    
    
# Sanity check
assert np.all(np.isclose(X_train_level2.mean(axis=0), [ 1.50148988,  1.38811989]))

27
Test R-squared for linreg is 0.485961 
Test R-squared for LightGBM is 0.274626
XGB R2: 0.307945144639
28
Test R-squared for linreg is 0.549966 
Test R-squared for LightGBM is 0.475513
XGB R2: 0.701825600073
29
Test R-squared for linreg is 0.796330 
Test R-squared for LightGBM is 0.607881
XGB R2: 0.850811037151
30
Test R-squared for linreg is 0.799710 
Test R-squared for LightGBM is 0.640794
XGB R2: 0.861103284419
31
Test R-squared for linreg is 0.843263 
Test R-squared for LightGBM is 0.688985
XGB R2: 0.82921810112
32
Test R-squared for linreg is 0.398219 
Test R-squared for LightGBM is 0.402381
XGB R2: 0.572297797322


AssertionError: 

In [66]:
alphas_to_try = np.linspace(0, 1, 1001)
best_alpha = 0
r2_train_simple_mix = 0
# YOUR CODE GOES HERE
for alpha in alphas_to_try:
    mix = alpha*X_train_level2[:,0]+(1-alpha)*X_train_level2[:,1]
    r2_alpha = r2_score(y_train_level2, mix)
    if r2_alpha > r2_train_simple_mix:
        best_alpha = alpha
        r2_train_simple_mix = r2_alpha
        
#best_alpha = # YOUR CODE GOES HERE
#r2_train_simple_mix = # YOUR CODE GOES HERE

print('Best alpha: %f; Corresponding r2 score on train: %f' % (best_alpha, r2_train_simple_mix))

Best alpha: 0.762000; Corresponding r2 score on train: 0.627492


In [75]:
test_preds = best_alpha * X_test_level2[:,0] + (1-best_alpha)* X_test_level2[:,1] # YOUR CODE GOES HERE
r2_test_simple_mix = r2_score(y_test, test_preds)# YOUR CODE GOES HERE

print('Test R-squared for simple mix is %f' % r2_test_simple_mix)

Test R-squared for simple mix is 0.782636


In [109]:
lr.fit(X_train_l2, y_train_level2)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [110]:
train_preds = lr.predict(X_train_l2) # YOUR CODE GOES HERE
r2_train_stacking = r2_score(y_train_level2, train_preds)# YOUR CODE GOES HERE

test_preds = lr.predict(X_test_l2)# YOUR CODE GOES HERE
r2_test_stacking = r2_score(y_test, test_preds)# YOUR CODE GOES HERE

print('Train R-squared for stacking is %f' % r2_train_stacking)
print('Test  R-squared for stacking is %f' % r2_test_stacking)

Train R-squared for stacking is 0.658969
Test  R-squared for stacking is 0.527451


In [113]:
rf = RandomForestRegressor( oob_score=True, verbose=1)
rf.fit(X_train_l2, y_train_level2)
train_pred_rf = rf.predict(X_train_l2)
train_r2_rf = r2_score(y_train_level2, train_pred_rf)
print("Train r2:",train_r2_rf)

test_pred_rf = rf.predict(X_test_l2)
r2_test_rf = r2_score(y_test, test_pred_rf)
print("Test r2:", r2_test_rf)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.5s finished
C:\Users\girishgupta\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:723: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


Train r2: 0.931955833147


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


Test r2: 0.913508533475


In [114]:
rf.feature_importances_

array([ 0.62342324,  0.23338146,  0.14319531])